In [ ]:
pip install requests

In [ ]:
pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import os

url = "https://thewaltdisneycompany.com/investor-relations/"

response = requests.get(url)
response.raise_for_status()  

soup = BeautifulSoup(response.text, 'html.parser')

pdf_links = soup.find_all('a', class_='pdf-file')

save_dir = "disney_reports"
os.makedirs(save_dir, exist_ok=True)

# Choose what to download: 'annual' or 'quarterly'
report_type = 'quarterly' 

for link in pdf_links:
    pdf_url = link.get('href')
    if pdf_url.endswith('.pdf'):
        link_text = link.get_text(strip=True).lower() 
        
        if (report_type == 'annual' and 'annual report' in link_text) or \
           (report_type == 'quarterly' and ('q1' in link_text or 'q2' in link_text or 'q3' in link_text or 'q4' in link_text or 'earnings report' in link_text)):
            pdf_response = requests.get(pdf_url)
            pdf_response.raise_for_status()
            
            file_name = pdf_url.split('/')[-1]
            save_path = os.path.join(save_dir, file_name)
            
            with open(save_path, 'wb') as f:
                f.write(pdf_response.content)
            print(f"Downloaded {file_name}")

print(f"All {report_type} have been downloaded.")


In [ ]:
pip install PyMuPDF

In [13]:
import fitz
import os

def extract_section(pdf_path, start_keywords, end_keywords):
    """
    Extracts sections from a PDF based on start and end keywords.

    :param pdf_path: Path to the PDF file.
    :param start_keywords: List of keywords indicating the start of the section.
    :param end_keywords: List of keywords indicating the end of the section.
    :return: Text of the extracted section.
    """
    doc = fitz.open(pdf_path)
    text = ""
    capture = False
    
    for page in doc:
        page_text = page.get_text()
        if any(keyword.lower() in page_text.lower() for keyword in start_keywords) and not capture:
            capture = True
        if any(keyword.lower() in page_text.lower() for keyword in end_keywords) and capture:
            text += page_text
            break
        if capture:
            text += page_text
    
    doc.close()
    return text

In [ ]:
pdf_dir = "disney_reports"
balance_sheet_keywords = ["balance sheet"]
cash_flows_keywords = ["cash flows", "statement of cash flows"]
income_statement_keywords = ["income statement", "statement of operations"]

for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, pdf_file)
        print(f"Extracting from {pdf_file}...")
        
        balance_sheet_section = extract_section(pdf_path, balance_sheet_keywords, cash_flows_keywords)
        print("Balance Sheet Section:")
        print(balance_sheet_section)